## Advent of code 2019 day 21-25
See https://adventofcode.com/

In [ ]:
# note that this notebook requires the .venv-pypy environment for pypy 3.9
# to activate it from a git bash shell: source .venv-pypy/Scripts/activate
# to generate its requirements: pip freeze > .venv-pypy-requirements.txt

import collections
import itertools
import re
import copy
import math
import sys
import time
import json
import heapq
import random
from sortedcontainers import SortedDict
#import cProfile

In [ ]:
# utility functions and version check

def get_line_groups(lines):
    '''return list of lists of lines, each separated by empty lines, ignores empty lines from start and end'''
    lines=list(lines)
    lines.append('') # add terminator
    res=[]
    group=[]
    for line in lines:
        line=line.strip()
        if len(line)>0:
            group.append(line)
        elif len(group)>0: # close group
            res.append(group)
            group=[]
    return res

class StopExecution(Exception):
    def _render_traceback_(self):
        pass

def exit():
    raise StopExecution()
    
print(f'python version: {sys.version}')
print(f'# start_ts={int(time.time())}') # supports ranking using an honor system, before starting include this line
# in the header of your solution (which should start with a line like # 2019 day 2), then whenever you want save
# a private leaderboard json file, and run python privaterank.py filename.json

In [ ]:
# 2019 day 22
# start_ts=1669513983
# (actual start was 1669447151, took 66832 secs of breaks before finishing)
# mv ~/Downloads/input* data_src/2019-day-22-input.txt
# big input file looks like: game step log of 100 lines
# idea: part 1 parse ..., then ...

sample2='''
deal into new stack
cut -2
deal with increment 7
cut 8
cut -4
deal with increment 7
cut 3
deal with increment 9
deal with increment 3
cut -1
'''

def last_word(s):
    '''returns (lcode, last word as int)'''    
    try:
        larg=int(s.split()[-1])
    except:
        larg=None
    for lcode, pref in enumerate(['deal into new stack', 'cut', 'deal with increment']):
        if s.startswith(pref):
            return (lcode, larg)
    assert False

def shuffle_increment(cards, larg):
    '''deal with increment'''
    assert larg>0
    res=[]
    while len(res)<len(cards):
        res.append(None)
    pos=0
    for x in cards:
        assert res[pos] is None
        res[pos]=x
        pos=(pos+larg)%len(res)
    return res

def shuffle(cards, lines):
    '''execute lines on cards, returning result, lines should be tuples of (lcode, larg)'''
    for lcode, larg in lines:
        if lcode==0:
            cards=list(reversed(cards))
        elif lcode==1:
            cards=cards[larg:]+cards[:larg]
        elif lcode==2:
            cards=shuffle_increment(cards, larg)
        else:
            assert False
    return cards

sample1=open('data_src/2019-day-22-input.txt').read()
lines=[ last_word(s) for s in sample1.splitlines() if len(s)>0 ]
#shuffle(list(range(10)), lines)
cards1=shuffle(list(range(10007)), lines)
print('part 1:', cards1.index(2019))

# part 1: 4096

In [ ]:
# part 2
# idea 1: reverse all operations based on a single position, run the list of lines backwards starting
# with position 2020 (this will give position x, which would, when run forward, end up in position 2020 after a 
# single run through), now repeat this until you find a cycle, then you only have to run the remaining number
# of times (perfect shuffling is hard, so there must be cycles) - unfortunately the cycle is apparently 
# too long to find quickly
# idea 2: putting all reversed operations into a formula that can be chained, and then multiplied by the 
# times to repeat the lines - unfortunately deal-with-increment cannot be reversed so elegantly
# idea 3: see below

def rev_shuffle_increment(n, larg, pos):
    '''reverse of pos=(larg*pos)%n'''
    while True:
        if pos%larg==0:
            return pos//larg
        pos+=n

def revshuffle(n, endpos, lines):
    '''reverse execute, returning the starting position that would end up in endpos'''
    pos=endpos
    for lcode, larg in reversed(lines):
        if lcode==0:
            pos=(-1*pos+n-1)%n
        elif lcode==1:
            pos=(pos+larg)%n
        elif lcode==2:
            pos=rev_shuffle_increment(n, larg, pos)
        else:
            assert False
    return pos

# test using cards1
#for endpos in range(10007):
#    startpos=revshuffle(10007, endpos, lines)
#    assert startpos==cards1[endpos]
# even simpler
print('check, should be 2019', revshuffle(10007, 4096, lines))

# find cycle - disabled, doesn't work / takes too long
#count=0
#pos=2020
#while True:
#    pos=revshuffle(119315717514047, pos, lines)
#    count-=1
#    if pos==2020:
#        print(f'ended on {pos=} in turns {count} and 0')
#        break
#    if count%100000000==0:
#        print(f'in progress, ended on {pos=} in turn {count}')


In [ ]:
# part 2 idea 3 A0

mys_cache={} # maps n to mySeries intermediate result, not threadsafe

# calculate x^n+x^(n-1)+..+x^0 % modul for large numbers
# mySeries(x, n)=mySeries(x, 1e6)*x^(n-1e6) + mySeries(x, n-1e6)
def mySeries(x: int, n: int, modul: int) -> int:
    global mys_cache
    mys_cache.clear()
    x=x%modul
    return mySeries1(x, n, modul)

def mySeries1(x: int, n: int, modul: int) -> int:
    global mys_cache
    if n==0:
        return 1
    assert n>0
    if n in mys_cache:
        return mys_cache[n]
    if n>1000000:
        n1=n//2
        n2=n-n1
        res=(mySeries1(x, n1, modul)*pow(x, n2, modul) + mySeries1(x, n2-1, modul)) % modul
        mys_cache[n]=res
        return res
    res=0
    mult=1
    for _ in range(n+1):
        res=(res+mult) % modul
        mult=(mult*x) % modul
    mys_cache[n]=res
    return res

In [ ]:
# part 2 idea 3 A1
# test mySeries

testms_x=173094589
testms_n=10132095
testms_mod=100
testms_res=1
for i in range(1, testms_n+1):
    testms_res=(testms_res+pow(testms_x, i, testms_mod)) % testms_mod
print(f'{testms_res}')
testms_res2=mySeries(testms_x, testms_n, testms_mod)
print(f'{testms_res2}')
assert testms_res==testms_res2


In [ ]:
# part 2 idea 3 A2
# idea: put the forward operations into a chainable formula, and then guess at a value that results in
# 2020, observing / searching for changes in the starting position that will move us in the right 
# direction

def fwdshuffle(n, spos, lines):
    '''forward execute, returning the end position resulting from specified starting pos'''
    pos=spos
    for lcode, larg in lines:
        if lcode==0:
            pos=(-1*pos+n-1)%n
        elif lcode==1:
            pos=(pos-larg)%n
        elif lcode==2:
            pos=(larg*pos)%n
        else:
            assert False
    return pos

def fwdshuffle2(n, spos, lines):
    '''forward execute, returning the end position resulting from specified starting pos,
    creating a cumulative formula'''
    a=1 # the formula to maintain is epos=a*spos+b
    b=0
    for lcode, larg in lines:
        if lcode==0:
            a= -a
            b= -b+n-1
        elif lcode==1:
            b-=larg
        elif lcode==2:
            a*=larg
            b*=larg
        else:
            assert False
    b=b%n
    #print(f'{a=}, {b=}')
    return (a*spos+b)%n, a, b

# to calculate: epos=(ta*spos+tb)%n
def fn_epos(spos, ta, tb, n):
    spos=spos%n
    res=(ta*spos+tb)%n
    return res

print('check, should be 4096', fwdshuffle2(10007, 2019, lines)[0])
for spos in range(10007):
    epos=fwdshuffle2(10007, spos, lines)[0]
    assert cards1.index(spos)==epos
# now test shuffling twice - five times
n=10007
cards2=cards1
for runtimes in range(2, 6):
    dummy, a, b=fwdshuffle2(n, 0, lines)
    ta=pow(a, runtimes, mod=n)
    tb=(mySeries(a, runtimes-1, modul=n)*b)%n
    cards2=shuffle(cards2, lines)
    for spos in range(10007):
        epos=fn_epos(spos, ta, tb, n)
        assert cards2.index(spos)==epos
print('multishuffling checked out')

# now for real part 2
n=119315717514047
runtimes=101741582076661
dummy, a, b=fwdshuffle2(n, 0, lines)
ta=pow(a, runtimes, mod=n)
tb=(mySeries(a, runtimes-1, modul=n)*b)%n
epos=2020

# analyse effect of varying spos on resulting epos
pos=n//2
epos0=fn_epos(pos-1, ta, tb, n)
epos1=fn_epos(pos, ta, tb, n)
epos2=fn_epos(pos+1, ta, tb, n)
diff0=min((epos0-epos1)%n, (epos1-epos0)%n)
diff2=min((epos2-epos1)%n, (epos1-epos2)%n)
td=diff0
assert diff2==td
print(f'{td=}') # increasing/decreasing spos by 1 will shift around spos by td, ca. half of n

In [ ]:
# part 2 idea 3 B
# filling xtd, which contains multiple steps of td and their effect
# within the modulo operation, in particular looking for small steps

def xtd_effect(xtd, td, i):
    i=i%n
    effect=min((td*i)%n, n-(td*i)%n)
    if effect not in xtd and effect!=0:
        xtd[effect]=i
    return effect
    
xtd=SortedDict() 
xtdn=2000000
for i in range(1, xtdn//2):
    xtd_effect(xtd, td, i)
for i in range(1, xtdn//2):
    xtd_effect(xtd, td, random.randrange(n))
bestval=n
while bestval>1:
    # of best keys add doubles and halves, differences and additions
    for i in range(1000):
        k=xtd.keys()[i]
        j=xtd[k]
        xtd_effect(xtd, td, 2*j)
        if j%2==0:
            xtd_effect(xtd, td, j//2)
        k2=xtd.keys()[i+1]
        j2=xtd[k2]
        xtd_effect(xtd, td, j+j2)
        xtd_effect(xtd, td, j2-j)
        xtd_effect(xtd, td, j-j2)
    for j in range(100000):
        xtd_effect(xtd, td, random.randrange(n))
    while len(xtd)>xtdn:
        j=xtd.keys()[-1] # remove worst items
        del xtd[j]
    if xtd.keys()[0]<bestval:
        bestval=xtd.keys()[0]
        print(f'progress, {bestval=}')
xtd.items()[:10]

In [ ]:
# part 2 idea 3 C
# we will repeatedly apply xtd elements to try to come as close as possible to epos

pos=n//2
count=0
bestdiff=n
while True:
    epos1=fn_epos(pos, ta, tb, n)
    if epos1==epos:
        print(f'found, spos: {pos}')
        break
    diff1=min((epos-epos1)%n, (epos1-epos)%n)
    count+=1
    if count%1000000==0:
        print(f'turn {count}, diff {diff1}')
    if diff1<bestdiff:
        bestdiff=diff1
        print(f'progress in turn {count}, diff {diff1}')
    i=xtd.bisect_left(diff1)
    if i>=len(xtd):
        i=len(xtd)-1
    elif i<0:
        i=0
    j=xtd.keys()[i]
    if j>diff1 and i>0:
        j=xtd.keys()[i-1]
    i=xtd[j]
    epos0=fn_epos(pos-i, ta, tb, n)
    epos2=fn_epos(pos+i, ta, tb, n)
    diff0=min((epos-epos1)%n, (epos-epos0)%n)
    diff2=min((epos-epos1)%n, (epos-epos2)%n)
    if diff0<diff2:
        pos=(pos-i)%n
    else:
        pos=(pos+i)%n

# part 2: 78613970589919

In [ ]:
# 2019 day 21
# start_ts=1668846011
# mv ~/Downloads/input* data_src/2019-day-21-input.txt
# big input file looks like: IntCode
# idea: part 1 parse ..., then ...

class Computer:
    def __init__(self):
        self.relbase=0
        self.i_ptr=0
        self.OFFSET_DIVS={1: 100, 2: 1000, 3: 10000}
        self.board=None

    def fetch_param(self, data, opcode, i, offset):
        mode=(opcode//self.OFFSET_DIVS[offset]) % 10
        param=data[i+offset]
        if mode==0: # position mode
            return data[param]
        elif mode==1: # immediate mode
            return param
        elif mode==2: # relative mode
            return data[param+self.relbase]
        else:
            print(f'invalid mode {mode}')
            assert False

    def store_param(self, data, opcode, i, offset, newval):
        mode=(opcode//self.OFFSET_DIVS[offset]) % 10
        param=data[i+offset]
        if mode==0: # position mode
            data[param]=newval
        elif mode==2: # relative mode
            data[param+self.relbase]=newval
        else:
            print(f'invalid mode {mode}')
            assert False

    def run_opcodes(self, data): # HALTS ON INPUT -1 !
        i=self.i_ptr
        halted=False
        while True:
            if data[i]%100==1: # add
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, a+b)
                i+=4
            elif data[i]%100==2: # mult
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, a*b)
                i+=4
            elif data[i]%100==3: # input
                a=self.get_input()
                if a==-1:
                    halted=True
                    break
                self.store_param(data, data[i], i, 1, a)
                i+=2
            elif data[i]%100==4: # output
                a=self.fetch_param(data, data[i], i, 1)
                self.put_output(a)
                i+=2
            elif data[i]%100==5: # jump-if-true
                a=self.fetch_param(data, data[i], i, 1)
                if a!=0:
                    i=self.fetch_param(data, data[i], i, 2)
                else:
                    i+=3
            elif data[i]%100==6: # jump-if-false
                a=self.fetch_param(data, data[i], i, 1)
                if a==0:
                    i=self.fetch_param(data, data[i], i, 2)
                else:
                    i+=3
            elif data[i]%100==7: # less-than
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, 1 if a<b else 0)
                i+=4
            elif data[i]%100==8: # equals
                a=self.fetch_param(data, data[i], i, 1)
                b=self.fetch_param(data, data[i], i, 2)
                self.store_param(data, data[i], i, 3, 1 if a==b else 0)
                i+=4
            elif data[i]%100==9: # relbase
                a=self.fetch_param(data, data[i], i, 1)
                self.relbase+=a
                i+=2
            elif data[i]%100==99: # stop
                halted=True
                break
            else:
                print(f'unknown instruction {data[i]} at position {i}')
                assert False
        self.i_ptr=i
        return

    def init_board(self, do_part):
        assert self.board is None
        self.board={} # maps (x,y) cell to ascii code
        self.do_part=do_part
        self.px=0
        self.py=0
        self.part1plan=None # used for both parts
        self.part1planpos=0
        self.part1res=None

    def get_input(self):
        if self.board is None:
            self.init_board()
        #if self.do_part==1 or self.do_part==2:
        c=self.part1plan[self.part1planpos]
        self.part1planpos+=1
        return ord(c)

    def put_output(self, a):
        if self.board is None:
            self.init_board()
        # if self.do_part==1: # same for both parts
        if a==10:
            self.px=0
            self.py+=1
        elif a<=255:
            self.board[ (self.px, self.py) ]=a
            self.px+=1
        else:
            self.part1res=a

    def print_painted(self, noprint=False):
        y_vals=[tup[1] for tup in self.board.keys()]
        x_vals=[tup[0] for tup in self.board.keys()]
        res=[]
        for y in range(min(y_vals), max(y_vals)+1):
            row=''
            for x in range(min(x_vals), max(x_vals)+1):
                c=self.board.get( (x, y) , -1)
                row+=chr(c) if c>=32 else '?'
            if not noprint:
                print(row)
            res.append(row)
        return res

# jumps if ground at 4 and not at 1 or 2 or 3
p1cmd1='''
OR A T
AND B T
AND C T
NOT T J
AND D J
WALK
'''

# jumps if ground at 4 and not at (1 and 2 and 3)
# except if after jump ground not at 1 and not at 4, ie except if ground not at 5 and not at 8,
# ie and there must be ground at 5 or 8
p2cmd1='''
OR A T
AND B T
AND C T
NOT T J
AND D J
AND E T
OR E T
OR H T
AND T J
RUN
'''

sample1=open('data_src/2019-day-21-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data0=[ int(s) for s in lines[0].split(',') ] # program
for do_part in [1,2]:
    print(f'{do_part=}')
    data=collections.defaultdict(lambda : 0)
    for i,v in enumerate(data0): # converted to large mem
        data[i]=v
    cmp=Computer()
    cmp.init_board(do_part)
    if do_part==1:
        cmp.part1plan=p1cmd1.replace('\n', '', 1)
    else:
        cmp.part1plan=p2cmd1.replace('\n', '', 1)
    cmp.run_opcodes(data)
    if do_part==1 or do_part==2:
        sample2_strs=cmp.print_painted()
        part1res=cmp.part1res
        print(f'part {do_part} {part1res=}')

# part 1: 19360288
# part 2: 1143814750


In [ ]:
# TEMPLATE
# 2019 day 6
# start_ts=RUN FIRST CELL TO GET TIME CODE BEFORE OPENING THE ASSIGNMENT
# mv ~/Downloads/input* data_src/2019-day-6-input.txt
# big input file looks like: 
# idea: part 1 parse ..., then ...

sample2='''

'''

sample1=open('data_src/2019-day-6-input.txt').read()
lines=[s for s in sample1.splitlines() if len(s)>0 ]
data=[ int(s) for s in lines[0].split(',') ]
groups=get_line_groups(lines)
data0=[ s.split() for s in lines ]
data0=[ [cmd, int(num), 0] for cmd, num in data0 ]
data=[ result.group(1, 2, 3, 4, 5, 6, 7) for s in lines if (result:= re.match(r'(\w+)\s*x=([\d\-]+)\.\.([\d\-]+),y=([\d\-]+)\.\.([\d\-]+),z=([\d\-]+)\.\.([\d\-]+)', s)) ]
data=[ (row[0], int(row[1]), int(row[2]), int(row[3]), int(row[4]), int(row[5]), int(row[6]) ) for row in data ]
# template, remove what's not needed